### Before you start

You need a gmaps API key to run this notebook. I didn't want to put it here in plain, so ask me directly.

In [1]:
API_KEY = "<fill in>"

In [2]:
import dispatcher

import pandas as pd
import numpy as np

from datetime import datetime as dt
import matplotlib.pyplot as plt

# maps viz
import gmaps
import gmaps.datasets

%pylab inline

dispatcher.configure(API_KEY) # Your Google API key

Populating the interactive namespace from numpy and matplotlib


#### The dispatcher package contains some useful functions that use the google maps API:

In [3]:
# convert and address to latitude and longitude
loc1 = dispatcher.address_to_latlon("1132 Euclid Ave, Berkeley, CA 94708")
print(loc1)
loc2 = dispatcher.address_to_latlon("50 Whitaker Ave, Berkeley, CA 94708")
print(loc2)

# get the time (in sec) to drive from one location to another
t = dispatcher.get_duration(loc1, loc2)
print("{:.2} minutes".format(t/60.))

{'lat': 37.8883437, 'lng': -122.2616888}
{'lat': 37.8902953, 'lng': -122.2558843}
7.3 minutes


In [4]:
# test addresses
addresses = [
    "1030 creston road, Berkeley, CA",
    "Tilden Regional Park Shasta Road at, Wildcat Canyon Rd, Berkeley, CA 94708",
    "2286 Cedar St, Berkeley, CA 94709",
    "950 Indian Rock Ave, Berkeley, CA 94707",
    "50 Whitaker Ave, Berkeley, CA 94708",
    "1132 Euclid Ave, Berkeley, CA 94708",
    "1 Centennial Dr, Berkeley, CA 94720",
    "101 Colusa Ave, El Cerrito, CA 94530",
    "30 Santa Barbara Rd, Berkeley, CA 94707",
    "41 Somerset Pl, Berkeley, CA 94707"
]

In [5]:
locations = [dispatcher.address_to_latlon(adr) for adr in addresses]

In [6]:
locations

[{'lat': 37.89470439999999, 'lng': -122.2584098},
 {'lat': 37.8936229, 'lng': -122.2454012},
 {'lat': 37.8787842, 'lng': -122.2649013},
 {'lat': 37.89214279999999, 'lng': -122.2729682},
 {'lat': 37.8902953, 'lng': -122.2558843},
 {'lat': 37.8883437, 'lng': -122.2616888},
 {'lat': 37.8793781, 'lng': -122.2467773},
 {'lat': 37.9074111, 'lng': -122.2874857},
 {'lat': 37.8960052, 'lng': -122.2707927},
 {'lat': 37.8947778, 'lng': -122.2740412}]

In [7]:
df = pd.DataFrame([{'lat': 37.89470439999999, 'lng': -122.2584098},
 {'lat': 37.8936229, 'lng': -122.2454012},
 {'lat': 37.8787842, 'lng': -122.2649013},
 {'lat': 37.89214279999999, 'lng': -122.2729682},
 {'lat': 37.8902953, 'lng': -122.2558843},
 {'lat': 37.8883437, 'lng': -122.2616888},
 {'lat': 37.8793781, 'lng': -122.2467773},
 {'lat': 37.9074111, 'lng': -122.2874857},
 {'lat': 37.8960052, 'lng': -122.2707927},
 {'lat': 37.8947778, 'lng': -122.2740412}])

In [8]:
df

,lat,lng
0,37.894704,-122.258410
1,37.893623,-122.245401
2,37.878784,-122.264901
3,37.892143,-122.272968
4,37.890295,-122.255884
5,37.888344,-122.261689
6,37.879378,-122.246777
7,37.907411,-122.287486
8,37.896005,-122.270793
9,37.894778,-122.274041


In [9]:
fig = gmaps.figure()

fig.add_layer(gmaps.symbol_layer(df, fill_color="red", stroke_color="red", scale=4))
fig

Figure(layout=FigureLayout(height='420px'))

### routing optimization experiments

This is a very simple solution for the travelling salesman problem with fixed starting point:
we pick for the next destination whichever point is the closest.
With N destination points, this algorithm makes N + N-1 + N-2 ... = N(N-1)/2 calls to the google maps directions API.

In [10]:
def travelling_salesman_NN(start_location, destinations):
    return find_NN_route(start_location, destinations, [], 0)

In [11]:
def find_NN_route(start_location, destinations, route, total_time):
    """tail-recursive function"""
    if len(destinations) == 0:
        return route, total_time
    min_dist = np.inf
    closest = None
    for p in destinations:
        dist = dispatcher.get_duration(start_location, p)
        if dist < min_dist:
            min_dist = dist
            closest = p
    print("Found new point in route")
    route.append(closest)
    total_time += min_dist
    return find_NN_route(closest, [p for p in destinations if p != closest], route, total_time)

In [12]:
start = df.loc[0].to_list()
destinations = df.iloc[1:].values.tolist()
route, time = travelling_salesman_NN(start, destinations)
print("Route found: {} min".format(time/60))

Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Route found: 42.8 min


In [13]:
route

[[37.8902953, -122.2558843],
 [37.8936229, -122.2454012],
 [37.8793781, -122.2467773],
 [37.8787842, -122.2649013],
 [37.89214279999999, -122.2729682],
 [37.8947778, -122.2740412],
 [37.8960052, -122.2707927],
 [37.8883437, -122.2616888],
 [37.9074111, -122.2874857]]

In [14]:
# careful: this makes N-1 calls to the google maps directions API
fig = gmaps.figure()
route_fig = gmaps.directions_layer(
        route[0], route[-1], waypoints=route[1:-1],
        travel_mode='DRIVING')
fig.add_layer(route_fig)
fig

Figure(layout=FigureLayout(height='420px'))

However, we usually need 2 constraints: on the start point and end point.

The following is a slightly modifed version of the previous algorithm where we bounce from start to end recursively.

In [15]:
def travelling_salesman_NN(start_location, end_location, destinations):
    route = [start_location] + [None]*len(destinations) + [end_location]
    route, time = _find_NN_route(destinations, route, 0)
    # reverse if need be
    if route[0] != start_location:
        route.reverse()
    return route, time


def _find_NN_route(destinations, route, total_time):
    """Recursively builds the route between the start point and end point"""
    if len(destinations) == 0:
        return route, total_time
    # find previous point and index of point to add
    start = None
    for i, p in enumerate(route):
        if p is None:
            start = route[i-1]
            break
    closest, min_dist = find_closest(start, destinations)
    print("Found new point in route")
    route[i] = destinations[closest]
    total_time += min_dist
    # we start from other side of the path next iteration
    route.reverse()
    return _find_NN_route([p for (i, p) in enumerate(destinations) if i != closest], route, total_time)

# helper function
def find_closest(start, destinations):
    min_dist = np.inf
    closest = None
    for i, p in enumerate(destinations):
        dist = dispatcher.get_duration(start, p)
        if dist < min_dist:
            min_dist = dist
            closest = i
    return closest, min_dist

In [16]:
start = df.iloc[0].to_list()
end = df.iloc[-1].to_list()
destinations = df.iloc[1:-1].values.tolist()
route, time = travelling_salesman_NN(start, end, destinations)
print("Route found: {} min".format(time/60))

Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Found new point in route
Route found: 39.18333333333333 min


In [17]:
fig = gmaps.figure()
route_fig = gmaps.directions_layer(
        route[0], route[-1], waypoints=route[1:-1],
        travel_mode='DRIVING')
fig.add_layer(route_fig)
fig

Figure(layout=FigureLayout(height='420px'))

All these functions can be found in the dispatcher package.

In [18]:
dispatcher.travelling_salesman_NN

<function dispatcher.optimization.travelling_salesman_NN(start_location, end_location, destinations)>